<a href="https://colab.research.google.com/github/MinMint01/VRSE/blob/main/VRSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
count=0
success=True
video=cv2.VideoCapture('/content/drive/MyDrive/VIDEOS/Input.mp4')
if not video.isOpened():
    print("Error opening video file!")
while success:
    success,image=video.read()
    if success:
        cv2.imwrite("/content/drive/MyDrive/VIDEOS/InputFrames/frame%d.jpg" % count,image)
        count+=1

In [ ]:
print("Enter the number of videos in the database: ")
n=int(input())

def split(x):
  count=0
  success=True
  video=cv2.VideoCapture('/content/drive/MyDrive/VIDEOS/DataBase/video%d.mp4' % x)
  if not video.isOpened():
    print("Error opening database video file : video%d!" % x)
  print("video",x,": \n")
  while success:
    success,image=video.read()
    if success:
      cv2.imwrite("/content/drive/MyDrive/VIDEOS/Temp%d/frame%d.jpg" % (x,count),image)
      count+=1
    else:
      break

for i in range(n):
  split(i)


Enter the number of videos in the database: 
4
video 0 : 

video 1 : 

video 2 : 

video 3 : 



In [ ]:
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms

resnet18 = models.resnet18(pretrained=True)

# Remove the final layer (the classification layer) from the network
feature_extractor = torch.nn.Sequential(*list(resnet18.children())[:-1])

# Define a function to extract features from an image using the ResNet-18 feature extractor
def extract_features(image_path, feature_extractor):
    # Load the image and apply the necessary transformations
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])
    image = transform(image).unsqueeze(0)

    # Extract features using the feature extractor
    with torch.no_grad():
        features = feature_extractor(image).squeeze()

    return features.numpy()

# Define a function to compare two images using cosine similarity
def compare_images(image1_path, image2_path, feature_extractor):
    # Extract features from both images
    features1 = extract_features(image1_path, feature_extractor)
    features2 = extract_features(image2_path, feature_extractor)

    # Calculate cosine similarity between the two feature vectors
    similarity = np.dot(features1, features2) / (np.linalg.norm(features1) * np.linalg.norm(features2))

    return similarity

# Example usage: compare two images using ResNet-18 features
image1_path = '/content/drive/MyDrive/VIDEOS/InputFrames/frame101.jpg'
image2_path = '/content/drive/MyDrive/VIDEOS/Temp0/frame833.jpg'
similarity = compare_images(image1_path, image2_path, feature_extractor)
print("Cosine similarity between the two images:", similarity)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Cosine similarity between the two images: 0.9985549


In [ ]:

import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import xlwt
from xlwt import Workbook
wb = Workbook()
sheet1 = wb.add_sheet('CSim')

resnet18 = models.resnet18(pretrained=True)

# Remove the final layer (the classification layer) from the network
feature_extractor = torch.nn.Sequential(*list(resnet18.children())[:-1])

# Define a function to extract features from an image using the ResNet-18 feature extractor
def extract_features(image_path, feature_extractor):
    # Load the image and apply the necessary transformations
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])
    image = transform(image).unsqueeze(0)

    # Extract features using the feature extractor
    with torch.no_grad():
        features = feature_extractor(image).squeeze()

    return features.numpy()

# Define a function to compare two images using cosine similarity
def compare_images(image1_path, image2_path, feature_extractor):
    # Extract features from both images
    features1 = extract_features(image1_path, feature_extractor)
    features2 = extract_features(image2_path, feature_extractor)

    # Calculate cosine similarity between the two feature vectors
    similarity = np.dot(features1, features2) / (np.linalg.norm(features1) * np.linalg.norm(features2))

    return similarity

# Example usage: compare two images using ResNet-18 features
ndb=int(input("Enter the number of frames in video 0: "))

image1_path = '/content/drive/MyDrive/VIDEOS/InputFrames/frame101.jpg'

for i in range(n):
  print(i)
  image2_path='/content/drive/MyDrive/VIDEOS/Temp0/frame%d.jpg' % i
  similarity = compare_images(image1_path, image2_path, feature_extractor)
  sheet1.write(i,0,similarity)
  print("Cosine similarity between the input image and frame%d in video 0:" % i, similarity)
wb.save('/content/drive/MyDrive/VIDEOS/Cosine Similarity.xls')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Enter the number of frames in video 0: 1572
0
Cosine similarity between the input image and frame0 in video 0: 0.99809754
1
Cosine similarity between the input image and frame1 in video 0: 0.9980881
2
Cosine similarity between the input image and frame2 in video 0: 0.99804133
3
Cosine similarity between the input image and frame3 in video 0: 0.99790317


In [ ]:
!pip install xlwt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 8.5 MB/s eta 0:00:00
